# Q 1 Algorithmic questions

In [24]:
# Import the packages that will be used
import heapq
from typing import List

In [50]:
# Wrapping up
def k_largest_num(arr: List, k: int) -> int:
    '''
    The solution is wrapped up into function k_largest_num()
    input: arr(the unsorted array), k(the rank of the target number)
    output: the k-th largest number in the unsorted array
    '''
    if not arr:
        print("The array is empty")
        return
    # Based on the unsorted array, establish the min-heap (can be transformed in linear time)
    heapq.heapify(arr) 
    # Pop out the element from the heap. For the min-heap, the k-th largest number is equivalent to the (n-k+1)-th smallest number
    for i in range(len(arr) - k + 1):
        target = heapq.heappop(arr)
    return target

In [51]:
# Develop several test cases and explain why you choose these test cases
# test_1: when the elements in the unsorted array are all distinct
test_1 = [3, 2, 1, 5, 6, 4]
k1 = 2
print(k_largest_num(test_1, k1))

# test_2: when there are repeated number in the array
test_2 = [2, 3, 4, 4, 4, 5, 6]
k2 = 4
print(k_largest_num(test_2, k2))

# test_3: when the input array is empty
test_3 = []
k3 = 0
print(k_largest_num(test_3, k3))

5
4
The array is empty
None


#### Analyze the time complexity of your algorithm
* Building the min-heap based on the unsorted array using heapq.heapify() takes O(n) time (linear time cost)
* The iteration part in the function, it takes O(nlogn) time. Inside of the iteration, we need to heapq.heappop() the root of the min-heap. This operation includes:
    * 1. Exchange the root element with the last element in the heap, and decrease the size of the min-heap by one;
    * 2. Min-heapify the current heap (adjust it to make sure it maintain the properties of a min-heap)
    * As this will iterate for (n-k+1) times and min-heapify takes O(logn) time, the iteration part will take O(nlogn) in total.
* Therefore, the algorithm will take O(nlogn+n) time.

# Q2 Multithreading pop

In [20]:
# Import packages
import queue
import timeit
import threading
import logging
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

# Function definition
def time_calculator(func):
    '''
        Calculate the execution time of a function
    '''
    def inner(*args, **kwargs):
        start_t = timeit.default_timer()
        #print("Start_time: ", start_t)
        func(*args, **kwargs)
        end_t = timeit.default_timer()
        #print("End_time:", end_t)
        print("Execution Time: {} seconds".format(end_t - start_t))
    return inner

# Define function for popping element out of the queue
def Queue_Pop_Thread(q, name, work_amount):
    #print(f"Thread {name}: starting")
    for count in range(work_amount):
        if not q.empty():
            q.get()
        else:
            print("WARNING: The queue is empty!")
            break
    #print(f"Thread {name}: finishing")
    
# Utilizing Threading
@time_calculator
def threading_pop(q, num_worker):
    threads = []
    for i in range(num_worker):
        #print(f"Create and start thread {i}")
        t = threading.Thread(target = Queue_Pop_Thread, args = (q, i, int(q.qsize() / num_worker)))
        threads.append(t)
        t.start()

    # Wait for the threads to finish
    for t in threads:
        t.join()
            
# Utilize ProcessExecutor (Overcomes GIL limitations)
@time_calculator
def process_executor(q, num_worker):
    with ProcessPoolExecutor(max_workers = num_worker) as executor:
        for i in range(q.qsize()):
            executor.submit(q.get())

__Utilize the Threading to achieve multithreading__
* __Observation__: 
  <br>Notice that when there are one thread, the time cost is 0.02989s; when there are five threads, the time cost is 0.01861932; and when there are 20 threads, the time cost is 0.056273s. That is, as the # of threads increases, the time cost will also increase.
  <br>This is resulted by GIL (General Interpreter Lock), which only allows one thread to run at one time and makes running in parallel impossible. Therefore, as the # of threads increases, the time cost will also increase (it also takes time to make allocation among each thread).
* __Quick question__: 
<br>For doing a queue poping, when allocating the tasks to each threads, we divide the task evenly to each task (see in the function Queue_Pop_Thread(), we have a parameter work amount equal to int(q.qsize()/num_worker)). Based on this allocation, will it be possible that the elements in queue are not all popped out? (i.e. q is not empty at the end of the multithreading pop)

In [25]:
# Generage a queue that have 10000 elements
# Construct a FIFO queue
q1 = queue.Queue(maxsize = 10000)
q2 = queue.Queue(maxsize = 10000)
q3 = queue.Queue(maxsize = 10000)
for i in range(10000):
    q1.put(i)
    q2.put(i)
    q3.put(i)

# Compare Efficiency using Threading
print("Threading with 1 thread")

threading_pop(q1, 1)
print("Threading with 5 threads")
threading_pop(q2, 5)
print("Threading with 20 threads")
threading_pop(q3, 20)

Threading with 1 thread
Execution Time: 0.02157360000001063 seconds
Threading with 5 threads
Execution Time: 0.01816570000005413 seconds
Threading with 20 threads
Execution Time: 0.02206069999999727 seconds


__Utilize the ProcessPoolExecutor to achieve multithreading__
* __Observation__: 
  <br> I have noticed that the time cost also increases as the # of the threads increases.
  <br> But I am very confused about it. As mentioned in class, ProcessPoolExecutor can overcome GIL. However, in this output, the time costs doesn't decrease when the # of threads increases.
  <br> Is there anything worng with the function "process_executor()"?


In [26]:
# Generage a queue that have 10000 elements
# Construct a FIFO queue
q4 = queue.Queue(maxsize = 10000)
q5 = queue.Queue(maxsize = 10000)
q6 = queue.Queue(maxsize = 10000)
for i in range(100):
    q4.put(i)
    q5.put(i)
    q6.put(i)

# Compare Efficiency using ThreadPoolExecutor
print("Multithreading with 1 thread")
process_executor(q4, 1)
print("Multithreading with 5 threads")
process_executor(q5, 5)
print("Multithreading with 20 threads")
process_executor(q6, 20)

Multithreading with 1 thread
Execution Time: 0.1567588999999998 seconds
Multithreading with 5 threads
Execution Time: 0.6690106999999443 seconds
Multithreading with 20 threads
Execution Time: 1.009322300000008 seconds
